In [1]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-6.3.0-posix-seh-rt_v5-rev1\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
% pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from xgboost.sklearn import XGBRegressor
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV

Populating the interactive namespace from numpy and matplotlib


C:\Users\Ambar Gupta\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Ambar Gupta\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
train = pd.read_csv('train.csv')

In [4]:
test = pd.read_csv('test.csv')

In [5]:
train.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,surcharge,fare_amount
0,AIX000345001,DST000401,NO,0.0,1.4,0.5,2015-04-20 04:18:25,2015-04-20 04:24:20,1,-74.003939,40.742894,1,N,-73.993369,40.734247,CRD,0.5,8.4
1,AIX000345002,DST000401,NO,0.0,1.0,0.5,2015-04-19 18:16:07,2015-04-19 18:22:06,3,-73.973864,40.752194,1,N,-73.958701,40.772533,CRD,0.0,8.5
2,AIX000345003,DST000401,NO,0.0,0.0,0.5,2015-04-06 08:04:32,2015-04-06 08:10:05,2,-73.954406,40.764420,1,N,-73.970780,40.758350,CSH,0.0,7.0
3,AIX000345004,DST000532,NO,0.0,1.8,0.5,2015-04-10 09:48:00,2015-04-10 10:00:00,2,-73.962345,40.767215,1,NaN,-73.975512,40.756867,CRD,0.0,11.3
4,AIX000345005,DST000401,NO,0.0,0.0,0.5,2015-04-15 13:12:48,2015-04-15 13:26:30,1,-74.004657,40.707434,1,N,-73.999369,40.721517,CSH,0.0,10.0


In [6]:
train.describe()

C:\Users\Ambar Gupta\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,tolls_amount,tip_amount,mta_tax,passenger_count,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,surcharge,fare_amount
count,1.658099e+06,1.492903e+06,1.658099e+06,1.658099e+06,1.608290e+06,1.624998e+06,1.658099e+06,1.653112e+06,1.649937e+06,1.558926e+06,1.658099e+06
mean,2.770844e-01,1.554779e+00,4.980460e-01,1.722504e+00,-7.258480e+01,3.998628e+01,1.035918e+00,-7.258127e+01,3.998414e+01,3.177296e-01,1.531551e+01
std,1.890095e+00,2.356677e+00,3.194602e-02,1.384761e+00,1.004539e+01,5.528833e+00,4.683282e-01,1.005460e+01,5.538230e+00,3.623541e-01,1.272185e+01
min,-1.175000e+01,0.000000e+00,-5.000000e-01,0.000000e+00,-9.908311e+01,-1.355188e+01,0.000000e+00,-1.190126e+02,-1.355188e+01,-1.000000e+00,0.000000e+00
25%,0.000000e+00,NaN,5.000000e-01,1.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN,NaN,8.300000e+00
50%,0.000000e+00,NaN,5.000000e-01,1.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN,NaN,1.150000e+01
75%,0.000000e+00,NaN,5.000000e-01,2.000000e+00,NaN,NaN,1.000000e+00,NaN,NaN,NaN,1.700000e+01
max,1.450090e+03,2.312000e+02,5.000000e-01,9.000000e+00,7.945679e+01,6.798689e+01,2.100000e+02,0.000000e+00,6.343189e+01,2.050000e+01,1.502890e+03


In [7]:
train['new_user'] = train['new_user'].fillna('NO')

In [8]:
train['new_user'].isnull().sum()

0

In [9]:
train['new_user'].value_counts()

NO     1657936
YES        163
Name: new_user, dtype: int64

In [10]:
train['new_user'] = pd.Series(np.where(train.new_user.values=='YES',1,0),train.index)

In [11]:
train['new_user'].value_counts()

0    1657936
1        163
Name: new_user, dtype: int64

In [12]:
train['new_user'].sum()

163

In [13]:
train.columns

Index(['TID', 'vendor_id', 'new_user', 'tolls_amount', 'tip_amount', 'mta_tax',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'rate_code',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'payment_type', 'surcharge', 'fare_amount'],
      dtype='object')

In [14]:
train['vendor_id'].isnull().sum()

0

In [15]:
train['tolls_amount'].isnull().sum()

0

In [16]:
train['tip_amount'].isnull().sum()

165196

In [17]:
train['tip_amount'].sum()

2321133.8700004644

In [18]:
test['tip_amount'].sum()

794436.4699998912

In [19]:
me = (2321133.8700004644+2321133.8700004644)/(len(train)+len(test))

In [20]:
print(me)

2.141359264950447


In [21]:
train['tip_amount'] = train['tip_amount'].fillna(me)
test['tip_amount'] = test['tip_amount'].fillna(me)

In [22]:
test['vendor_id'].isnull().sum()
test['tolls_amount'].isnull().sum()

0

In [23]:
test['new_user'] = test['new_user'].fillna('NO')

In [24]:
test['new_user'].isnull().sum()

0

In [25]:
len(test)

509808

In [26]:
test['new_user'] = pd.Series(np.where(test.new_user.values=='YES',1,0),test.index)

In [27]:
test['new_user'].value_counts()

0    509633
1       175
Name: new_user, dtype: int64

In [28]:
train['new_user'].value_counts()

0    1657936
1        163
Name: new_user, dtype: int64

In [29]:
train.columns

Index(['TID', 'vendor_id', 'new_user', 'tolls_amount', 'tip_amount', 'mta_tax',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'rate_code',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'payment_type', 'surcharge', 'fare_amount'],
      dtype='object')

In [30]:
train['mta_tax'].isnull().sum()

0

In [31]:
test['mta_tax'].isnull().sum()

0

In [32]:
train['dropoff_datetime'].isnull().sum()

0

In [33]:
test['dropoff_datetime'].isnull().sum()

0

In [34]:
test['passenger_count'].isnull().sum()

0

In [35]:
test['rate_code'].isnull().sum()

0

In [36]:
train['store_and_fwd_flag'].value_counts()

N    993926
Y     16051
Name: store_and_fwd_flag, dtype: int64

In [37]:
test['store_and_fwd_flag'].value_counts()

N    323708
     181953
Y      4147
Name: store_and_fwd_flag, dtype: int64

In [38]:
train.columns

Index(['TID', 'vendor_id', 'new_user', 'tolls_amount', 'tip_amount', 'mta_tax',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'rate_code',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'payment_type', 'surcharge', 'fare_amount'],
      dtype='object')

In [39]:
test['payment_type'].isnull().sum()

0

In [40]:
train['surcharge'].isnull().sum()

99173

In [41]:
test['surcharge'].isnull().sum()

30727

In [42]:
train['surcharge'].sum()

495316.95

In [43]:
test['surcharge'].sum()

151321.69

In [44]:
me = (495316.95+151321.69)/(len(train)+len(test))

In [45]:
print(me)

0.2982778504797484


In [46]:
train['surcharge'] = train['surcharge'].fillna(me)
test['surcharge'] = test['surcharge'].fillna(me)

In [47]:
train['surcharge'].isnull().sum()
test['surcharge'].isnull().sum()

0

In [48]:
train.columns

Index(['TID', 'vendor_id', 'new_user', 'tolls_amount', 'tip_amount', 'mta_tax',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'rate_code',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'payment_type', 'surcharge', 'fare_amount'],
      dtype='object')

In [49]:
df = pd.DataFrame()

In [50]:
df = train['payment_type']

In [51]:
df = pd.get_dummies(df)
df.head()

,CRD,CSH,DIS,NOC,UNK
0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0


In [52]:
train = train.drop('payment_type',axis=1)

In [53]:
train = pd.concat([train,df],axis=1)

In [54]:
train.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,surcharge,fare_amount,CRD,CSH,DIS,NOC,UNK
0,AIX000345001,DST000401,0,0.0,1.4,0.5,2015-04-20 04:18:25,2015-04-20 04:24:20,1,-74.003939,...,N,-73.993369,40.734247,0.5,8.4,1.0,0.0,0.0,0.0,0.0
1,AIX000345002,DST000401,0,0.0,1.0,0.5,2015-04-19 18:16:07,2015-04-19 18:22:06,3,-73.973864,...,N,-73.958701,40.772533,0.0,8.5,1.0,0.0,0.0,0.0,0.0
2,AIX000345003,DST000401,0,0.0,0.0,0.5,2015-04-06 08:04:32,2015-04-06 08:10:05,2,-73.954406,...,N,-73.970780,40.758350,0.0,7.0,0.0,1.0,0.0,0.0,0.0
3,AIX000345004,DST000532,0,0.0,1.8,0.5,2015-04-10 09:48:00,2015-04-10 10:00:00,2,-73.962345,...,NaN,-73.975512,40.756867,0.0,11.3,1.0,0.0,0.0,0.0,0.0
4,AIX000345005,DST000401,0,0.0,0.0,0.5,2015-04-15 13:12:48,2015-04-15 13:26:30,1,-74.004657,...,N,-73.999369,40.721517,0.0,10.0,0.0,1.0,0.0,0.0,0.0


In [55]:
df = pd.DataFrame()
df = test['payment_type']
df = pd.get_dummies(df)
df.head()

,CRD,CSH,DIS,NOC,UNK
0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0


In [56]:
test = test.drop('payment_type',axis=1)
test = pd.concat([test,df],axis=1)
test.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,surcharge,CRD,CSH,DIS,NOC,UNK
0,AJX000119871,DST000481,0,0.0,0.00,0.5,2016-05-14 13:42:33,2016-05-14 13:45:43,1,-73.971794,...,1,Y,-73.972488,40.791576,0.0,0.0,1.0,0.0,0.0,0.0
1,AJX000119872,DST000543,0,0.0,1.26,0.5,2016-05-04 01:24:39,2016-05-04 01:29:21,2,-73.992210,...,1,,-73.987244,40.720276,0.5,1.0,0.0,0.0,0.0,0.0
2,AJX000119873,DST000543,0,0.0,0.00,0.5,2016-05-28 15:12:21,2016-05-28 15:25:02,6,-73.991753,...,1,N,-73.979828,40.761417,0.0,0.0,1.0,0.0,0.0,0.0
3,AJX000119874,DST000543,0,0.0,0.00,0.5,2016-05-02 21:08:40,2016-05-02 21:34:24,4,-73.957588,...,1,N,-73.919868,40.695366,0.5,0.0,1.0,0.0,0.0,0.0
4,AJX000119875,DST000543,0,0.0,0.00,0.5,2016-05-09 17:00:33,2016-05-09 17:18:45,1,-73.970985,...,1,N,-73.991409,40.731861,0.0,0.0,1.0,0.0,0.0,0.0


In [57]:
train.isnull().sum()

TID                        0
vendor_id                  0
new_user                   0
tolls_amount               0
tip_amount                 0
mta_tax                    0
pickup_datetime            0
dropoff_datetime           0
passenger_count            0
pickup_longitude       49809
pickup_latitude        33101
rate_code                  0
store_and_fwd_flag    648122
dropoff_longitude       4987
dropoff_latitude        8162
surcharge                  0
fare_amount                0
CRD                        0
CSH                        0
DIS                        0
NOC                        0
UNK                        0
dtype: int64

In [58]:
test.isnull().sum()

TID                       0
vendor_id                 0
new_user                  0
tolls_amount              0
tip_amount                0
mta_tax                   0
pickup_datetime           0
dropoff_datetime          0
passenger_count           0
pickup_longitude      15339
pickup_latitude       10095
rate_code                 0
store_and_fwd_flag        0
dropoff_longitude      1584
dropoff_latitude       2563
surcharge                 0
CRD                       0
CSH                       0
DIS                       0
NOC                       0
UNK                       0
dtype: int64

In [59]:
train = train.drop('store_and_fwd_flag',axis=1)

In [60]:
train.isnull().sum()

TID                      0
vendor_id                0
new_user                 0
tolls_amount             0
tip_amount               0
mta_tax                  0
pickup_datetime          0
dropoff_datetime         0
passenger_count          0
pickup_longitude     49809
pickup_latitude      33101
rate_code                0
dropoff_longitude     4987
dropoff_latitude      8162
surcharge                0
fare_amount              0
CRD                      0
CSH                      0
DIS                      0
NOC                      0
UNK                      0
dtype: int64

In [61]:
def convert_vendorid(s):
    if '401' in s:
        return 0
    else:
        return int(s[-1])

In [62]:
train['vendor_id'] = train['vendor_id'].apply(lambda x:convert_vendorid(x))
test['vendor_id'] = test['vendor_id'].apply(lambda x:convert_vendorid(x))

In [63]:
train.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,rate_code,dropoff_longitude,dropoff_latitude,surcharge,fare_amount,CRD,CSH,DIS,NOC,UNK
0,AIX000345001,0,0,0.0,1.4,0.5,2015-04-20 04:18:25,2015-04-20 04:24:20,1,-74.003939,...,1,-73.993369,40.734247,0.5,8.4,1.0,0.0,0.0,0.0,0.0
1,AIX000345002,0,0,0.0,1.0,0.5,2015-04-19 18:16:07,2015-04-19 18:22:06,3,-73.973864,...,1,-73.958701,40.772533,0.0,8.5,1.0,0.0,0.0,0.0,0.0
2,AIX000345003,0,0,0.0,0.0,0.5,2015-04-06 08:04:32,2015-04-06 08:10:05,2,-73.954406,...,1,-73.970780,40.758350,0.0,7.0,0.0,1.0,0.0,0.0,0.0
3,AIX000345004,2,0,0.0,1.8,0.5,2015-04-10 09:48:00,2015-04-10 10:00:00,2,-73.962345,...,1,-73.975512,40.756867,0.0,11.3,1.0,0.0,0.0,0.0,0.0
4,AIX000345005,0,0,0.0,0.0,0.5,2015-04-15 13:12:48,2015-04-15 13:26:30,1,-74.004657,...,1,-73.999369,40.721517,0.0,10.0,0.0,1.0,0.0,0.0,0.0


In [64]:
test.columns

Index(['TID', 'vendor_id', 'new_user', 'tolls_amount', 'tip_amount', 'mta_tax',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'rate_code',
       'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude',
       'surcharge', 'CRD', 'CSH', 'DIS', 'NOC', 'UNK'],
      dtype='object')

In [65]:
test = test.drop('store_and_fwd_flag',axis=1)

In [66]:
train.columns

Index(['TID', 'vendor_id', 'new_user', 'tolls_amount', 'tip_amount', 'mta_tax',
       'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'pickup_longitude', 'pickup_latitude', 'rate_code', 'dropoff_longitude',
       'dropoff_latitude', 'surcharge', 'fare_amount', 'CRD', 'CSH', 'DIS',
       'NOC', 'UNK'],
      dtype='object')

In [67]:
def convert_datetime(X):
    X['pickup_datetime'] = X['pickup_datetime'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    X['dropoff_datetime'] = X['dropoff_datetime'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))
    return X

In [68]:
train = convert_datetime(train)
test = convert_datetime(test)

In [69]:
train.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,rate_code,dropoff_longitude,dropoff_latitude,surcharge,fare_amount,CRD,CSH,DIS,NOC,UNK
0,AIX000345001,0,0,0.0,1.4,0.5,2015-04-20 04:18:25,2015-04-20 04:24:20,1,-74.003939,...,1,-73.993369,40.734247,0.5,8.4,1.0,0.0,0.0,0.0,0.0
1,AIX000345002,0,0,0.0,1.0,0.5,2015-04-19 18:16:07,2015-04-19 18:22:06,3,-73.973864,...,1,-73.958701,40.772533,0.0,8.5,1.0,0.0,0.0,0.0,0.0
2,AIX000345003,0,0,0.0,0.0,0.5,2015-04-06 08:04:32,2015-04-06 08:10:05,2,-73.954406,...,1,-73.970780,40.758350,0.0,7.0,0.0,1.0,0.0,0.0,0.0
3,AIX000345004,2,0,0.0,1.8,0.5,2015-04-10 09:48:00,2015-04-10 10:00:00,2,-73.962345,...,1,-73.975512,40.756867,0.0,11.3,1.0,0.0,0.0,0.0,0.0
4,AIX000345005,0,0,0.0,0.0,0.5,2015-04-15 13:12:48,2015-04-15 13:26:30,1,-74.004657,...,1,-73.999369,40.721517,0.0,10.0,0.0,1.0,0.0,0.0,0.0


In [71]:
def cal(X):
    df = pd.DataFrame()
    df['diff'] = X['dropoff_datetime'] - X['pickup_datetime']
    df['diff'] = df['diff'].apply(lambda x:divmod(x.days * 86400 + x.seconds, 60))
    df['diff'] = df['diff'].apply(lambda x:abs(x[0]+(x[1]/60)))
    return df['diff']

In [72]:
train['total_jrny'] = cal(train)

In [73]:
train[train['total_jrny']<0]

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,dropoff_longitude,dropoff_latitude,surcharge,fare_amount,CRD,CSH,DIS,NOC,UNK,total_jrny


In [74]:
test['total_jrny'] = cal(test)

In [76]:
train.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,dropoff_longitude,dropoff_latitude,surcharge,fare_amount,CRD,CSH,DIS,NOC,UNK,total_jrny
0,AIX000345001,0,0,0.0,1.4,0.5,2015-04-20 04:18:25,2015-04-20 04:24:20,1,-74.003939,...,-73.993369,40.734247,0.5,8.4,1.0,0.0,0.0,0.0,0.0,5.916667
1,AIX000345002,0,0,0.0,1.0,0.5,2015-04-19 18:16:07,2015-04-19 18:22:06,3,-73.973864,...,-73.958701,40.772533,0.0,8.5,1.0,0.0,0.0,0.0,0.0,5.983333
2,AIX000345003,0,0,0.0,0.0,0.5,2015-04-06 08:04:32,2015-04-06 08:10:05,2,-73.954406,...,-73.970780,40.758350,0.0,7.0,0.0,1.0,0.0,0.0,0.0,5.550000
3,AIX000345004,2,0,0.0,1.8,0.5,2015-04-10 09:48:00,2015-04-10 10:00:00,2,-73.962345,...,-73.975512,40.756867,0.0,11.3,1.0,0.0,0.0,0.0,0.0,12.000000
4,AIX000345005,0,0,0.0,0.0,0.5,2015-04-15 13:12:48,2015-04-15 13:26:30,1,-74.004657,...,-73.999369,40.721517,0.0,10.0,0.0,1.0,0.0,0.0,0.0,13.700000


In [75]:
test.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,...,rate_code,dropoff_longitude,dropoff_latitude,surcharge,CRD,CSH,DIS,NOC,UNK,total_jrny
0,AJX000119871,1,0,0.0,0.00,0.5,2016-05-14 13:42:33,2016-05-14 13:45:43,1,-73.971794,...,1,-73.972488,40.791576,0.0,0.0,1.0,0.0,0.0,0.0,3.166667
1,AJX000119872,3,0,0.0,1.26,0.5,2016-05-04 01:24:39,2016-05-04 01:29:21,2,-73.992210,...,1,-73.987244,40.720276,0.5,1.0,0.0,0.0,0.0,0.0,4.700000
2,AJX000119873,3,0,0.0,0.00,0.5,2016-05-28 15:12:21,2016-05-28 15:25:02,6,-73.991753,...,1,-73.979828,40.761417,0.0,0.0,1.0,0.0,0.0,0.0,12.683333
3,AJX000119874,3,0,0.0,0.00,0.5,2016-05-02 21:08:40,2016-05-02 21:34:24,4,-73.957588,...,1,-73.919868,40.695366,0.5,0.0,1.0,0.0,0.0,0.0,25.733333
4,AJX000119875,3,0,0.0,0.00,0.5,2016-05-09 17:00:33,2016-05-09 17:18:45,1,-73.970985,...,1,-73.991409,40.731861,0.0,0.0,1.0,0.0,0.0,0.0,18.200000


In [77]:
train = train.drop(['pickup_datetime','dropoff_datetime'],axis=1)

In [78]:
train.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,passenger_count,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,surcharge,fare_amount,CRD,CSH,DIS,NOC,UNK,total_jrny
0,AIX000345001,0,0,0.0,1.4,0.5,1,-74.003939,40.742894,1,-73.993369,40.734247,0.5,8.4,1.0,0.0,0.0,0.0,0.0,5.916667
1,AIX000345002,0,0,0.0,1.0,0.5,3,-73.973864,40.752194,1,-73.958701,40.772533,0.0,8.5,1.0,0.0,0.0,0.0,0.0,5.983333
2,AIX000345003,0,0,0.0,0.0,0.5,2,-73.954406,40.764420,1,-73.970780,40.758350,0.0,7.0,0.0,1.0,0.0,0.0,0.0,5.550000
3,AIX000345004,2,0,0.0,1.8,0.5,2,-73.962345,40.767215,1,-73.975512,40.756867,0.0,11.3,1.0,0.0,0.0,0.0,0.0,12.000000
4,AIX000345005,0,0,0.0,0.0,0.5,1,-74.004657,40.707434,1,-73.999369,40.721517,0.0,10.0,0.0,1.0,0.0,0.0,0.0,13.700000


In [79]:
test = test.drop(['pickup_datetime','dropoff_datetime'],axis=1)
test.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,passenger_count,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,surcharge,CRD,CSH,DIS,NOC,UNK,total_jrny
0,AJX000119871,1,0,0.0,0.00,0.5,1,-73.971794,40.797230,1,-73.972488,40.791576,0.0,0.0,1.0,0.0,0.0,0.0,3.166667
1,AJX000119872,3,0,0.0,1.26,0.5,2,-73.992210,40.713123,1,-73.987244,40.720276,0.5,1.0,0.0,0.0,0.0,0.0,4.700000
2,AJX000119873,3,0,0.0,0.00,0.5,6,-73.991753,40.744305,1,-73.979828,40.761417,0.0,0.0,1.0,0.0,0.0,0.0,12.683333
3,AJX000119874,3,0,0.0,0.00,0.5,4,-73.957588,40.717602,1,-73.919868,40.695366,0.5,0.0,1.0,0.0,0.0,0.0,25.733333
4,AJX000119875,3,0,0.0,0.00,0.5,1,-73.970985,40.763931,1,-73.991409,40.731861,0.0,0.0,1.0,0.0,0.0,0.0,18.200000


In [80]:
train.isnull().sum()

TID                      0
vendor_id                0
new_user                 0
tolls_amount             0
tip_amount               0
mta_tax                  0
passenger_count          0
pickup_longitude     49809
pickup_latitude      33101
rate_code                0
dropoff_longitude     4987
dropoff_latitude      8162
surcharge                0
fare_amount              0
CRD                      0
CSH                      0
DIS                      0
NOC                      0
UNK                      0
total_jrny               0
dtype: int64

In [81]:
test.isnull().sum()

TID                      0
vendor_id                0
new_user                 0
tolls_amount             0
tip_amount               0
mta_tax                  0
passenger_count          0
pickup_longitude     15339
pickup_latitude      10095
rate_code                0
dropoff_longitude     1584
dropoff_latitude      2563
surcharge                0
CRD                      0
CSH                      0
DIS                      0
NOC                      0
UNK                      0
total_jrny               0
dtype: int64

In [82]:
from math import cos, asin, sqrt
def distance(lat1, lon1, lat2, lon2):
    if(np.isnan(lat1) or np.isnan(lon1) or np.isnan(lat2) or np.isnan(lon2)):
        return 0
    else:
        p = 0.017453292519943295     #Pi/180
        a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
        return 12742 * asin(sqrt(a))

In [83]:
train['distance'] = train.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

In [84]:
train['distance'].sum()

54214293.659065835

In [85]:
test['distance'] = test.apply(lambda row:distance(row['pickup_latitude'],row['pickup_longitude'],row['dropoff_latitude'],row['dropoff_longitude']),axis=1)

In [86]:
test['distance'].sum()

17819280.67577609

In [87]:
me = (54214293.659065835+17819280.67577609)/(len(train)+len(test))
print(me)

33.2272437585385


In [88]:
me

33.2272437585385

In [89]:
def update_distance(x,me):
    if x==0:
        return me
    else:
        return x

In [90]:
train['distance'] = train['distance'].apply(lambda x:update_distance(x,me))

In [91]:
test['distance'] = test['distance'].apply(lambda x:update_distance(x,me))

In [92]:
train.isnull().sum()

TID                      0
vendor_id                0
new_user                 0
tolls_amount             0
tip_amount               0
mta_tax                  0
passenger_count          0
pickup_longitude     49809
pickup_latitude      33101
rate_code                0
dropoff_longitude     4987
dropoff_latitude      8162
surcharge                0
fare_amount              0
CRD                      0
CSH                      0
DIS                      0
NOC                      0
UNK                      0
total_jrny               0
distance                 0
dtype: int64

In [93]:
train.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,passenger_count,pickup_longitude,pickup_latitude,rate_code,...,dropoff_latitude,surcharge,fare_amount,CRD,CSH,DIS,NOC,UNK,total_jrny,distance
0,AIX000345001,0,0,0.0,1.4,0.5,1,-74.003939,40.742894,1,...,40.734247,0.5,8.4,1.0,0.0,0.0,0.0,0.0,5.916667,1.310554
1,AIX000345002,0,0,0.0,1.0,0.5,3,-73.973864,40.752194,1,...,40.772533,0.0,8.5,1.0,0.0,0.0,0.0,0.0,5.983333,2.597243
2,AIX000345003,0,0,0.0,0.0,0.5,2,-73.954406,40.764420,1,...,40.758350,0.0,7.0,0.0,1.0,0.0,0.0,0.0,5.550000,1.535378
3,AIX000345004,2,0,0.0,1.8,0.5,2,-73.962345,40.767215,1,...,40.756867,0.0,11.3,1.0,0.0,0.0,0.0,0.0,12.000000,1.598049
4,AIX000345005,0,0,0.0,0.0,0.5,1,-74.004657,40.707434,1,...,40.721517,0.0,10.0,0.0,1.0,0.0,0.0,0.0,13.700000,1.628146


In [94]:
test.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,passenger_count,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,surcharge,CRD,CSH,DIS,NOC,UNK,total_jrny,distance
0,AJX000119871,1,0,0.0,0.00,0.5,1,-73.971794,40.797230,1,-73.972488,40.791576,0.0,0.0,1.0,0.0,0.0,0.0,3.166667,0.631338
1,AJX000119872,3,0,0.0,1.26,0.5,2,-73.992210,40.713123,1,-73.987244,40.720276,0.5,1.0,0.0,0.0,0.0,0.0,4.700000,0.898759
2,AJX000119873,3,0,0.0,0.00,0.5,6,-73.991753,40.744305,1,-73.979828,40.761417,0.0,0.0,1.0,0.0,0.0,0.0,12.683333,2.151694
3,AJX000119874,3,0,0.0,0.00,0.5,4,-73.957588,40.717602,1,-73.919868,40.695366,0.5,0.0,1.0,0.0,0.0,0.0,25.733333,4.027715
4,AJX000119875,3,0,0.0,0.00,0.5,1,-73.970985,40.763931,1,-73.991409,40.731861,0.0,0.0,1.0,0.0,0.0,0.0,18.200000,3.959392


In [95]:
train = train.drop(['pickup_longitude','pickup_latitude','dropoff_longitude',
                   'dropoff_latitude'],axis=1)

In [96]:
train.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,passenger_count,rate_code,surcharge,fare_amount,CRD,CSH,DIS,NOC,UNK,total_jrny,distance
0,AIX000345001,0,0,0.0,1.4,0.5,1,1,0.5,8.4,1.0,0.0,0.0,0.0,0.0,5.916667,1.310554
1,AIX000345002,0,0,0.0,1.0,0.5,3,1,0.0,8.5,1.0,0.0,0.0,0.0,0.0,5.983333,2.597243
2,AIX000345003,0,0,0.0,0.0,0.5,2,1,0.0,7.0,0.0,1.0,0.0,0.0,0.0,5.550000,1.535378
3,AIX000345004,2,0,0.0,1.8,0.5,2,1,0.0,11.3,1.0,0.0,0.0,0.0,0.0,12.000000,1.598049
4,AIX000345005,0,0,0.0,0.0,0.5,1,1,0.0,10.0,0.0,1.0,0.0,0.0,0.0,13.700000,1.628146


In [97]:
test = test.drop(['pickup_longitude','pickup_latitude','dropoff_longitude',
                   'dropoff_latitude'],axis=1)
test.head()

,TID,vendor_id,new_user,tolls_amount,tip_amount,mta_tax,passenger_count,rate_code,surcharge,CRD,CSH,DIS,NOC,UNK,total_jrny,distance
0,AJX000119871,1,0,0.0,0.00,0.5,1,1,0.0,0.0,1.0,0.0,0.0,0.0,3.166667,0.631338
1,AJX000119872,3,0,0.0,1.26,0.5,2,1,0.5,1.0,0.0,0.0,0.0,0.0,4.700000,0.898759
2,AJX000119873,3,0,0.0,0.00,0.5,6,1,0.0,0.0,1.0,0.0,0.0,0.0,12.683333,2.151694
3,AJX000119874,3,0,0.0,0.00,0.5,4,1,0.5,0.0,1.0,0.0,0.0,0.0,25.733333,4.027715
4,AJX000119875,3,0,0.0,0.00,0.5,1,1,0.0,0.0,1.0,0.0,0.0,0.0,18.200000,3.959392


In [168]:
train.to_csv('updated_train.csv',index=False)
test.to_csv('updated_test.csv',index=False)

In [99]:
features = ['vendor_id','tolls_amount', 'tip_amount', 'mta_tax','passenger_count','rate_code','surcharge',
            'total_jrny', 'CRD','CSH', 'DIS', 'NOC', 'UNK', 'distance']

In [115]:
df1 = train.sample(frac=0.01)

In [116]:
len(df1)

16581

In [129]:
X_new = df1[features]
y_new = np.sqrt(df1['fare_amount'])

In [130]:
# parameter testing first
param_test1 = {
 'max_depth':[3,5,7,9],
 'min_child_weight':[1,3,5]
}
gsearch1 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=2,
 min_child_weight=1, gamma=0.0, colsample_bylevel=0.85 , colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test1,n_jobs=4,iid=False, cv=10)
gsearch1.fit(X_new,y_new)
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 9, 'min_child_weight': 3}, 0.9726829952875035)

In [131]:
param_test2 = {
 'max_depth':[9,10,11,12],
 'min_child_weight':[2,3,4]
}
gsearch2 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=2,
 min_child_weight=1, gamma=0.0, colsample_bylevel=0.85 , colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test2,n_jobs=4,iid=False, cv=10)
gsearch2.fit(X_new,y_new)
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 9, 'min_child_weight': 3}, 0.9726829952875035)

In [133]:
param_test3 = {
 'gamma':[0.0,0.003,0.005,0.007]
}
gsearch3 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=9,
 min_child_weight=3, gamma=0.0, colsample_bylevel=0.85 , colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test3,n_jobs=4,iid=False, cv=10)
gsearch3.fit(X_new,y_new)
gsearch3.best_params_, gsearch3.best_score_

({'gamma': 0.005}, 0.9727370540281546)

In [135]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=9,
 min_child_weight=3, gamma=0.005, colsample_bylevel=0.85 , colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test4,n_jobs=4,iid=False, cv=10)
gsearch4.fit(X_new,y_new)
gsearch4.best_params_, gsearch4.best_score_

({'colsample_bytree': 0.6, 'subsample': 0.7}, 0.9725523449120909)

In [136]:
param_test4 = {
 'subsample':[0.65,0.70,0.75],
 'colsample_bytree':[0.55,0.60,0.65]
}
gsearch4 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=9,
 min_child_weight=3, gamma=0.005, colsample_bylevel=0.85 , colsample_bytree=0.85,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=1,reg_alpha=0,reg_lambda=1),
 param_grid = param_test4,n_jobs=4,iid=False, cv=10)
gsearch4.fit(X_new,y_new)
gsearch4.best_params_, gsearch4.best_score_

({'colsample_bytree': 0.6, 'subsample': 0.7}, 0.9725523449120909)

In [137]:
param_test5 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch5 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=9,
 min_child_weight=3, gamma=0.005, colsample_bylevel=0.85 , colsample_bytree=0.6,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=0.7,reg_alpha=0,reg_lambda=1),
 param_grid = param_test5,n_jobs=4,iid=False, cv=10)
gsearch5.fit(X_new,y_new)
gsearch5.best_params_, gsearch5.best_score_

({'reg_alpha': 0.01}, 0.9728233652127208)

In [138]:
param_test6 = {
 'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
}
gsearch6 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=9,
 min_child_weight=3, gamma=0.005, colsample_bylevel=0.85 , colsample_bytree=0.6,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=0.7,reg_alpha=0,reg_lambda=1),
 param_grid = param_test6,n_jobs=4,iid=False, cv=10)
gsearch6.fit(X_new,y_new)
gsearch6.best_params_, gsearch6.best_score_

({'reg_alpha': 0.01}, 0.9728233652127208)

In [141]:
param_test7 = {
 'colsample_bytree':[0.5,0.6,0.7]
}
gsearch7 = GridSearchCV(estimator = XGBRegressor(learning_rate =0.1, n_estimators=100, max_depth=9,
 min_child_weight=3, gamma=0.005, colsample_bylevel=0.85 , colsample_bytree=0.6,max_delta_step=0,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=0.7,reg_alpha=0.01,reg_lambda=1),
 param_grid = param_test7,n_jobs=4,iid=False, cv=10)
gsearch7.fit(X_new,y_new)
gsearch7.best_params_, gsearch7.best_score_

({'colsample_bytree': 0.6}, 0.9728233652127208)

In [154]:
xgbmodel = XGBRegressor(learning_rate =0.01, n_estimators=500, max_depth=9,
 min_child_weight=3, gamma=0.005, colsample_bylevel=0.60 , colsample_bytree=0.6,
 objective= 'reg:linear', nthread=4, scale_pos_weight=1, seed=27,subsample=0.7,reg_alpha=0.01)

In [155]:
X = train[features]
y = np.sqrt(train['fare_amount'])

In [145]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

In [156]:
xgbmodel.fit(X,y)

XGBRegressor(base_score=0.5, colsample_bylevel=0.6, colsample_bytree=0.6,
       gamma=0.005, learning_rate=0.01, max_delta_step=0, max_depth=9,
       min_child_weight=3, missing=None, n_estimators=500, nthread=4,
       objective='reg:linear', reg_alpha=0.01, reg_lambda=1,
       scale_pos_weight=1, seed=27, silent=True, subsample=0.7)

In [146]:
linmodel = LinearRegression(normalize=True)

In [147]:
linmodel.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [157]:
arr = xgbmodel.feature_importances_
arr

array([ 0.0591002 ,  0.07340491,  0.17585714,  0.03237212,  0.03537254,
        0.05266957,  0.07478348,  0.22665347,  0.02620098,  0.02504136,
        0.00718479,  0.01979467,  0.00672257,  0.1848422 ], dtype=float32)

In [149]:
X_test = test[features]

In [158]:
pred = xgbmodel.predict(X_test)
pred = np.square(pred)

In [151]:
pred_lin = linmodel.predict(X_test)
pred_lin = np.square(pred_lin)

In [152]:
pred[0],pred_lin[0]

(4.9146967, 10.397367579920683)

In [167]:
submission = pd.DataFrame(data=[], columns=['TID', 'fare_amount'])
submission.TID = test.TID; submission.fare_amount = x
submission.to_csv('combined1.csv', index=False)
submission.head()

,TID,fare_amount
0,AJX000119871,4.921615
1,AJX000119872,7.663007
2,AJX000119873,10.636740
3,AJX000119874,19.708366
4,AJX000119875,15.345366


In [160]:
a = pd.read_csv('xgboost.csv')

In [161]:
a = a['fare_amount']

In [162]:
b = pd.read_csv('xgboost1.csv')

In [163]:
b = b['fare_amount']

In [164]:
x = (a*0.8)+(b*0.2)

In [165]:
x.head()

0     4.921615
1     7.663007
2    10.636740
3    19.708366
4    15.345366
Name: fare_amount, dtype: float64